In [1]:
!pip install lightgbm==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.1.0
    Uninstalling lightgbm-4.1.0:
      Successfully uninstalled lightgbm-4.1.0


In [2]:
#@title Load coordinates nearest points

import time
import plotly.express as px
import pandas as pd
from scipy.stats import shapiro
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import train_test_split
from lightgbm.sklearn import LGBMRegressor
import pickle
from sklearn.pipeline import Pipeline

coor=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/input_files/distan_lat42.22lon-8.63p2R1Km.csv")
px.set_mapbox_access_token("pk.eyJ1IjoiZ3JhbmFudHVpbiIsImEiOiJja3B4dGU4OTkwMTFmMm9ycnNhMjJvaGJqIn0.VWzx_PkD9A5cSUVsn_ijCA")
px.scatter_mapbox(coor, hover_data=['distance'],lat='lat', lon='lon',color='distance', title="Nearest points",
                           color_continuous_scale=px.colors.cyclical.IceFire,)

In [3]:
#@title Load station and meteorological model (Forecast D0). Concatenate station and meteorological model. Meteorological model score

station = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/input_files/LEVXY2018Y2022.csv",
                          usecols = ["time","tempd_o"],
                          parse_dates=["time"]).set_index("time")

#Shapiro test normality
print("Shapiro test normality")
sta, p = shapiro(station.tempd_o.sample(500))

print("\np value:",p)
#significance level alpha=0.01
alpha = 0.01
if p>alpha:
  print("Accept null hypothesis: normal distribution variable\n")
else:
  print("Reject null Hypothesis. Alternative hypothesis variable no normal distribution variable\n")

print("Observed dew temperature statistics")
display((round(station["tempd_o"]-273.15,2)).describe())

#Load
mody2018 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/input_files/lat42.22lon-8.63p2R1KmD0Y2018.csv",parse_dates=["time"])
mody2019 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/input_files/lat42.22lon-8.63p2R1KmD0Y2019.csv",parse_dates=["time"])
mody2020 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/input_files/lat42.22lon-8.63p2R1KmD0Y2020.csv",parse_dates=["time"])
mody2021 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/input_files/lat42.22lon-8.63p2R1KmD0Y2021.csv",parse_dates=["time"])
mody2022 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/input_files/lat42.22lon-8.63p2R1KmD0Y2022.csv",parse_dates=["time"])
model = pd.concat([mody2018,mody2019,mody2020,mody2021,mody2022]).drop(columns=["Unnamed: 0"]).set_index("time")

#concat station model
df_all = pd.concat([station,model],axis=1).dropna()
df_all








Shapiro test normality

p value: 5.703465717488143e-07
Reject null Hypothesis. Alternative hypothesis variable no normal distribution variable

Observed dew temperature statistics


count    89934.000000
mean        10.557875
std          4.641619
min        -13.150000
25%          7.850000
50%         10.850000
75%         13.850000
max         25.850000
Name: tempd_o, dtype: float64

,tempd_o,dir0,snow_prec0,snowlevel0,mod0,wind_gust0,mslp0,temp0,rh0,visibility0,...,cape1,cin1,cfh1,cfl1,cfm1,cft1,HGT5001,HGT8501,T5001,T8501
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 01:00:00,280.0,194.487213,4.700000e-12,902.739380,2.913697,8.417547,103084.335938,281.546753,0.909289,24077.052734,...,23.129999,-15.010574,0.141546,0.000000,0.0,0.141546,5590.513672,1558.336060,250.594376,273.468506
2018-01-01 02:00:00,281.0,188.501129,0.000000e+00,932.149353,2.754451,9.908416,103076.046875,281.170319,0.941909,24059.664062,...,23.860001,-20.136265,0.100000,0.000000,0.0,0.100000,5598.377441,1556.657349,251.014557,273.805298
2018-01-01 03:00:00,281.0,179.663055,0.000000e+00,921.418823,2.745423,4.196273,103044.367188,281.068939,0.991956,24047.859375,...,57.099998,-15.279875,0.300000,0.300000,0.0,0.300000,5604.513672,1557.682495,251.309189,273.476990
2018-01-01 04:00:00,282.0,188.336029,0.000000e+00,946.878601,2.701216,11.246392,103070.609375,282.164124,0.948673,24043.777344,...,38.160000,-14.909769,0.200000,0.500000,0.0,0.500000,5611.291992,1557.819946,251.368500,273.654633
2018-01-01 05:00:00,282.0,211.158997,0.000000e+00,1017.428345,3.090386,10.534806,103014.820312,282.710999,0.937205,24045.267578,...,71.970001,-6.809992,0.400000,0.000000,0.0,0.400000,5613.609863,1563.999146,251.327194,274.155701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 19:00:00,285.0,197.595871,0.000000e+00,1995.864990,11.100198,18.938650,101692.445312,287.660614,0.935921,19832.320312,...,27.980000,-0.210500,0.100000,0.765625,0.3,0.765625,5612.094238,1493.944946,256.804199,280.074890
2022-12-30 20:00:00,285.0,195.637833,0.000000e+00,1991.257568,11.492834,19.293737,101713.359375,287.651611,0.931984,24232.105469,...,29.920000,-0.436152,0.100000,0.750000,0.1,0.750000,5616.190918,1496.136475,257.134003,279.967987
2022-12-30 21:00:00,285.0,192.479736,0.000000e+00,1978.222412,11.132333,18.849682,101740.812500,287.520111,0.934648,24232.353516,...,32.889999,-0.463516,0.200000,0.609375,0.0,0.609375,5618.303223,1496.986084,257.348755,279.805054


In [4]:
#@title Add time variables. Machine learning algorithm score. Save algorithm

start_time = time.time()

df_all["hour"] = df_all.index.hour
df_all["month"] = df_all.index.month
df_all["dayofyear"] = df_all.index.dayofyear
df_all["weekofyear"] = df_all.index.isocalendar().week.astype(int)
df_all.iloc[:,1:].columns

#random seed 1
np.random.seed(1)

# X and Y
Y = df_all.tempd_o
X = df_all.iloc[:,1:]

# loop random
Median_AE_ml = []
Mean_AE_ml =[]
MSE_ml = []
R2_ml = []

#20 samples to test algorithm
#random seed 1
np.random.seed(1)
print("Machine learning results")
for t in range (0,4):
  #split variables
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.15, )

  #machine learning model
  ml_model = LGBMRegressor().fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)

  #Score calculation
  Mean_AE_ml.append(round(mean_absolute_error(y_test, y_pred),2))
  Median_AE_ml.append(round(median_absolute_error(y_test, y_pred),2))
  MSE_ml.append (round(mean_squared_error(y_test, y_pred),2))
  R2_ml.append(round(r2_score(y_test, y_pred),2))

display(pd.DataFrame({"MAE":Mean_AE_ml,"MSE":MSE_ml,"R2":R2_ml,"Median_AE":Median_AE_ml}))

#show results sample 200
result = pd.DataFrame({"y_pred":y_pred,"y_test":y_test.values}).set_index(y_test.index)
result["dif_ml"]=result['y_pred']-result['y_test']
print("Comparing differences machine learning meteorological model")
display(result["dif_ml"].describe())



#Shapiro test normality machine learning model
print("Shapiro test normality differences machine learning model")
sta, p = shapiro(result.dif_ml.sample(500))

print("\np value:",p)
#significance level alpha=0.01
alpha = 0.01
if p>alpha:
  print("Accept null hypothesis: normal distribution variable\n")
else:
  print("Reject null Hypothesis. Alternative hypothesis variable no normal distribution variable\n")


#download algorithm
x_var = X.columns
score = {"MAE_ml":Mean_AE_ml[-1]}
estimators = [('ml_model', ml_model)]
pipe = Pipeline(estimators)

al_file={"x_var":x_var,"coor":coor,"pipe":pipe,"score":score}
pickle.dump(al_file, open("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/algorithms/llmtempd_LEVX_d0.al", 'wb'))

#time
end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", round(execution_time/60,2), "minutes")

Machine learning results


,MAE,MSE,R2,Median_AE
0,0.95,1.57,0.93,0.75
1,0.94,1.59,0.93,0.74
2,0.94,1.59,0.93,0.73
3,0.92,1.53,0.93,0.72


Comparing differences machine learning meteorological model


count    6200.000000
mean        0.007822
std         1.236102
min        -5.283840
25%        -0.748814
50%        -0.034743
75%         0.686430
max         8.050384
Name: dif_ml, dtype: float64

Shapiro test normality differences machine learning model

p value: 1.9054736629699198e-13
Reject null Hypothesis. Alternative hypothesis variable no normal distribution variable

Execution time: 0.17 minutes


In [5]:
result

,y_pred,y_test,dif_ml
time,,,
2022-03-17 21:00:00,274.962058,267.0,7.962058
2022-02-21 01:00:00,278.936284,280.0,-1.063716
2022-04-08 07:00:00,284.313427,284.0,0.313427
2022-08-30 11:00:00,287.945762,287.0,0.945762
2021-06-25 22:00:00,287.486464,285.0,2.486464
...,...,...,...
2018-04-19 12:00:00,285.419062,286.0,-0.580938
2018-11-01 08:00:00,279.241257,282.0,-2.758743
2018-01-18 13:00:00,282.455854,281.0,1.455854
